In [ ]:
# Mounting Azure Blob Storage to a Databricks filesystem

dbutils.fs.mount(
    source='wasbs://.....@......blob.core.windows.net', # container@storageacc
    mount_point='/mnt/......', #container
    extra_configs={
        'fs.azure.account.key.mystorage2397.blob.core.windows.net': dbutils.secrets.get('.....', '......') # your secret scope's name (azure databricks), your secret's name (azure key vault)
        }
)

True

In [ ]:
%fs
ls "/mnt/electric-vehicle-data"

path,name,size,modificationTime
dbfs:/mnt/electric-vehicle-data/raw-data/,raw-data/,0,0
dbfs:/mnt/electric-vehicle-data/transformed-data/,transformed-data/,0,0


In [ ]:
# Read data from CSV files in the "/mnt/raw-data/" directory using Spark DataFrame

df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/electric-vehicle-data/raw-data/*.csv")

In [ ]:
# Convert Spark DataFrame to Pandas DataFrame
import pandas as pd

df2 = df.toPandas()
df2

,VIN,County,City,State,Postal_Code,Model_Year,Make,Model,Electric_Vehicle_Type,Clean_Alternative_Fuel_Vehicle_Eligibility,Electric_Range,Base_MSRP,Legislative_District,DOL_Vehicle_ID,Vehicle_Location,Electric_Utility,2020_Census_Tract
0,SADHD2S10K,King,Kent,WA,98030.0,2019,JAGUAR,I-PACE,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,234.0,0.0,33.0,108869071,POINT (-122.199755 47.37483),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
1,LPSED3KA7N,Kitsap,Port Orchard,WA,98366.0,2022,POLESTAR,PS2,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,26.0,203047582,POINT (-122.639265 47.5373),PUGET SOUND ENERGY INC,5.303509e+10
2,WB523CF09R,King,Sammamish,WA,98075.0,2024,BMW,IX,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,41.0,256112482,POINT (-122.03309 47.58153),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
3,WP1AE2A2XG,King,Bellevue,WA,98005.0,2016,PORSCHE,CAYENNE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,14.0,0.0,41.0,228791958,POINT (-122.16085 47.624515),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
4,KM8KRDAF9P,Clark,Washougal,WA,98671.0,2023,HYUNDAI,IONIQ 5,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,18.0,255402899,POINT (-122.35465 45.58359),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,5.301104e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173528,5YJ3E1EA0P,King,Redmond,WA,98052.0,2023,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,48.0,227238091,POINT (-122.12302 47.67668),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
173529,5YJXCBE22H,Snohomish,Snohomish,WA,98296.0,2017,TESLA,MODEL X,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,200.0,0.0,1.0,113289473,POINT (-122.15134 47.8851158),PUGET SOUND ENERGY INC,5.306105e+10
173530,1C4RJXR65R,King,Kent,WA,98031.0,2024,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21.0,0.0,33.0,257440403,POINT (-122.2012521 47.3931814),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
173531,5UXKT0C50G,King,Seattle,WA,98199.0,2016,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,14.0,0.0,36.0,114351471,POINT (-122.394185 47.639195),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10


In [ ]:
print(df2.dtypes)

VIN                                            object
County                                         object
City                                           object
State                                          object
Postal_Code                                   float64
Model_Year                                      int32
Make                                           object
Model                                          object
Electric_Vehicle_Type                          object
Clean_Alternative_Fuel_Vehicle_Eligibility     object
Electric_Range                                float64
Base_MSRP                                     float64
Legislative_District                          float64
DOL_Vehicle_ID                                  int32
Vehicle_Location                               object
Electric_Utility                               object
2020_Census_Tract                             float64
dtype: object


In [ ]:
# Change data types
df2['Postal_Code'] = df2['Postal_Code'].astype('object')
df2['Electric_Range'] = df2['Electric_Range'].astype('Int32')
df2['Base_MSRP'] = df2['Base_MSRP'].astype('Int32')
df2['Legislative_District'] = df2['Legislative_District'].astype('Int32')
df2['2020_Census_Tract'] = df2['2020_Census_Tract'].astype('object')

In [ ]:
print(df2.dtypes)

VIN                                           object
County                                        object
City                                          object
State                                         object
Postal_Code                                   object
Model_Year                                     int32
Make                                          object
Model                                         object
Electric_Vehicle_Type                         object
Clean_Alternative_Fuel_Vehicle_Eligibility    object
Electric_Range                                 Int32
Base_MSRP                                      Int32
Legislative_District                           Int32
DOL_Vehicle_ID                                 int32
Vehicle_Location                              object
Electric_Utility                              object
2020_Census_Tract                             object
dtype: object


In [ ]:
# Calculate the number of missing values for each column
df2.isnull().sum()

VIN                                             0
County                                          5
City                                            5
State                                           0
Postal_Code                                     5
Model_Year                                      0
Make                                            0
Model                                           0
Electric_Vehicle_Type                           0
Clean_Alternative_Fuel_Vehicle_Eligibility      0
Electric_Range                                  1
Base_MSRP                                       1
Legislative_District                          376
DOL_Vehicle_ID                                  0
Vehicle_Location                               10
Electric_Utility                                5
2020_Census_Tract                               5
dtype: int64

In [ ]:
# Drop rows with missing values from df2
df2 = df2.dropna()

In [ ]:
df2.isnull().sum()

VIN                                           0
County                                        0
City                                          0
State                                         0
Postal_Code                                   0
Model_Year                                    0
Make                                          0
Model                                         0
Electric_Vehicle_Type                         0
Clean_Alternative_Fuel_Vehicle_Eligibility    0
Electric_Range                                0
Base_MSRP                                     0
Legislative_District                          0
DOL_Vehicle_ID                                0
Vehicle_Location                              0
Electric_Utility                              0
2020_Census_Tract                             0
dtype: int64

In [ ]:
# Calculate number of rows
num_rows = df2.shape[0]
num_rows

173151

In [ ]:
# Calculate summary statistics for numeric columns
df2.describe()

,Model_Year,Electric_Range,Base_MSRP,Legislative_District,DOL_Vehicle_ID
count,173151.000000,173151.000000,173151.000000,173151.000000,1.731510e+05
mean,2020.436989,60.135304,1110.335747,29.146987,2.188368e+08
std,2.994179,92.640670,8501.184324,14.871933,7.658982e+07
min,1997.000000,0.000000,0.000000,1.000000,4.385000e+03
25%,2018.000000,0.000000,0.000000,18.000000,1.802026e+08
50%,2022.000000,0.000000,0.000000,33.000000,2.271138e+08
75%,2023.000000,83.000000,0.000000,42.000000,2.530245e+08
max,2024.000000,337.000000,845000.000000,49.000000,4.792548e+08


In [ ]:
# Distribution of electric vehicle types

# Groupby the 'Electric_Vehicle_Type' column and count the number of occurrences for each type
ev_type_distribution = df2.groupby('Electric_Vehicle_Type').size()

# Sort the groups by count in descending order
ev_type_distribution = ev_type_distribution.sort_values(ascending=False)

# Show the resulting grouped DataFrame
ev_type_distribution

Electric_Vehicle_Type
Battery Electric Vehicle (BEV)            135364
Plug-in Hybrid Electric Vehicle (PHEV)     37787
dtype: int64

In [ ]:
# Top 10 electric vehicle models

# Group by 'Model' column and count occurrences
top_10_models = df2.groupby('Model').size()

# Sort the groups by count in descending order
top_10_models = top_10_models.sort_values(ascending=False)

# Filter the first 10 rows from the dataframe
top_10_models = top_10_models.head(10)
top_10_models

Model
MODEL Y           34456
MODEL 3           29776
LEAF              13388
MODEL S            7718
BOLT EV            6647
MODEL X            5628
VOLT               4832
ID.4               3811
WRANGLER           3277
MUSTANG MACH-E     3269
dtype: int64

In [ ]:
# Top 10 Counties by Electric Vehicle Count

# Group by 'County' column and count occurrences
top_10_counties = df2.groupby('County').size()

# Sort the groups by count in descending order
top_10_counties = top_10_counties.sort_values(ascending=False)

# Filter the first 10 rows from the dataframe
top_10_counties = top_10_counties.head(10)
top_10_counties

County
King         90326
Snohomish    20473
Pierce       13508
Clark        10154
Thurston      6235
Kitsap        5692
Spokane       4473
Whatcom       4146
Benton        2108
Skagit        1888
dtype: int64

In [ ]:
# Top 10 Cities by Count of Vehicles

# Group by 'City' column and count occurrences
top_10_cities = df2.groupby('City').size()

# Sort the groups by count in descending order
top_10_cities = top_10_cities.sort_values(ascending=False)

# Filter the first 10 rows from the dataframe
top_10_cities = top_10_cities.head(10)
top_10_cities

City
Seattle      28856
Bellevue      8728
Redmond       6288
Vancouver     6038
Bothell       5658
Kirkland      5222
Sammamish     5095
Renton        4871
Olympia       4174
Tacoma        3647
dtype: int64

In [ ]:
# Top 10 Manufacturer by Count of Vehicles

# Group by 'Make' column and count occurrences
top_10_manufacturer = df2.groupby('Make').size()

# Sort the groups by count in descending order
top_10_manufacturer = top_10_manufacturer.sort_values(ascending=False)

# Filter the first 10 rows from the dataframe
top_10_manufacturer = top_10_manufacturer.head(10)
top_10_manufacturer

Make
TESLA         77624
NISSAN        13992
CHEVROLET     13460
FORD           8893
BMW            7450
KIA            7240
TOYOTA         6026
VOLKSWAGEN     4876
JEEP           4331
HYUNDAI        4244
dtype: int64

In [ ]:
# Top 10 Electric Vehicle Manufacturers by Average Electric Range

# Group by 'Make' column and calculate the average of 'Electric_Range' column on df2
average_range = df2.groupby('Make')['Electric_Range'].mean()

# Sort the result in descending order
average_range = average_range.sort_values(ascending=False)

# Filter the first 10 rows from the dataframe
top_10_average_range = average_range.head(10)
top_10_average_range

Make
JAGUAR                  204.105727
WHEEGO ELECTRIC CARS         100.0
TH!NK                        100.0
CHEVROLET                94.756538
FIAT                     85.621723
NISSAN                   80.812679
TESLA                    79.204125
SMART                    62.339416
AZURE DYNAMICS                56.0
AUDI                      51.30447
Name: Electric_Range, dtype: Float64

In [ ]:
# Average Electric Range by Electric Vehicle Type

# Group by 'Electric_Vehicle_Type' column and calculate the average of 'Electric_Range'
average_range = df2.groupby('Electric_Vehicle_Type')['Electric_Range'].mean()

# Sort the result in descending order
average_range = average_range.sort_values(ascending=False)
average_range

Electric_Vehicle_Type
Battery Electric Vehicle (BEV)            68.327443
Plug-in Hybrid Electric Vehicle (PHEV)    30.788684
Name: Electric_Range, dtype: Float64

In [ ]:
# Group the data by 'Electric_Vehicle_Type'
grouped_df = df2.groupby('Electric_Vehicle_Type')

# Calculate the average electric range and maximum price for each electric vehicle type
grouped_df1 = grouped_df.agg({'Electric_Range': 'mean', 'Base_MSRP': 'max'})
grouped_df1

,Electric_Range,Base_MSRP
Electric_Vehicle_Type,,
Battery Electric Vehicle (BEV),68.327443,110950
Plug-in Hybrid Electric Vehicle (PHEV),30.788684,845000


In [ ]:
# Top 10 Electric Vehicle Models by Average Electric Range

# Group by 'Model' column and calculate the average of 'Electric_Range'
average_range = df2.groupby('Model')['Electric_Range'].mean()

# Sort the result in descending order
average_range = average_range.sort_values(ascending=False)

# Filter the first 10 rows from the dataframe
top_10_average_range = average_range.head(10)
top_10_average_range

Model
KONA             258.0
ROADSTER    234.130435
I-PACE      204.105727
MODEL S     177.432107
BOLT EV     155.861893
MODEL X     140.019723
E-TRON      129.586654
MODEL 3     110.607536
E-GOLF      107.411268
RAV4         102.54717
Name: Electric_Range, dtype: Float64

In [ ]:
# Create a Spark DataFrame from df2
df2_spark = spark.createDataFrame(df2)

df2_spark.show()

+----------+---------+-----------------+-----+-----------+----------+---------+--------+---------------------+------------------------------------------+--------------+---------+--------------------+--------------+--------------------+--------------------+-----------------+
|       VIN|   County|             City|State|Postal_Code|Model_Year|     Make|   Model|Electric_Vehicle_Type|Clean_Alternative_Fuel_Vehicle_Eligibility|Electric_Range|Base_MSRP|Legislative_District|DOL_Vehicle_ID|    Vehicle_Location|    Electric_Utility|2020_Census_Tract|
+----------+---------+-----------------+-----+-----------+----------+---------+--------+---------------------+------------------------------------------+--------------+---------+--------------------+--------------+--------------------+--------------------+-----------------+
|SADHD2S10K|     King|             Kent|   WA|    98030.0|      2019|   JAGUAR|  I-PACE| Battery Electric ...|                      Clean Alternative...|           234|       

In [ ]:
# Write df2_spark dataframe to the specified location as CSV files, overwriting existing files
df2_spark.write.mode("overwrite").option("header",'true').csv("/mnt/electric-vehicle-data/transformed-data")